In [2]:
import pandas as pd
import numpy
import itertools
import csv
import glob

In [3]:
# Make a list of the file names I want to iterate through using glob
ranafilenames = glob.glob('../rana2brad/*outfiles/*stats.txt')
print ranafilenames

['../rana2brad/clust_91_outfiles/clust_91_stats.txt', '../rana2brad/clust_90_outfiles/clust_90_stats.txt', '../rana2brad/rana2brad_outfiles/rana2brad_stats.txt', '../rana2brad/clust_86_outfiles/clust_86_stats.txt', '../rana2brad/clust_87_outfiles/clust_87_stats.txt', '../rana2brad/clust_81_outfiles/clust_81_stats.txt', '../rana2brad/clust_80_outfiles/clust_80_stats.txt', '../rana2brad/clust_84_outfiles/clust_84_stats.txt', '../rana2brad/clust_93_outfiles/clust_93_stats.txt', '../rana2brad/clust_95_outfiles/clust_95_stats.txt', '../rana2brad/clust_88_outfiles/clust_88_stats.txt', '../rana2brad/clust_89_outfiles/clust_89_stats.txt']


In [110]:
def get_file_list(directory):
    return glob.glob(directory)

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'var  sum_var' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#            print line
        elif '## Final' in line:
            endline=counter
#            print name,endline
#            print line
        else:
            continue
    infile.close()
    return [varline,endline]

def slice_data(start,end,filename):
    snp_dist=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end-3):
        lines2 = lines.strip().split()
#        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        snp_dist.append(lines2)
    infile.close()
    return snp_dist
        

def pd_conversion(filename):
    nums = get_line_numbers(filename)
    snp_dist = slice_data(nums[0],nums[1],filename)
    snpdist_labels=['number','variable','sum_var','pis', 'sum_pis', 'clust_threshold']
    df_snpdist = pd.DataFrame.from_records(snp_dist, columns=snpdist_labels)
    return df_snpdist  

def main():
    directory='../epiddrad/*outfiles/*stats.txt'
    #filename='../epi2brad/clust_92_outfiles/clust_92_stats.txt'
    #directory=filename
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv(filename+"_snpdist.csv")
    return dfs

print main()

../epiddrad/clust_86_outfiles/clust_86_stats.txt
../epiddrad/clust_87_outfiles/clust_87_stats.txt
../epiddrad/clust_81_outfiles/clust_81_stats.txt
../epiddrad/clust_80_outfiles/clust_80_stats.txt
../epiddrad/clust_85_outfiles/epiddrad-nov29-lm10_stats.txt
../epiddrad/clust_84_outfiles/clust_84_stats.txt
../epiddrad/clust_82_outfiles/clust_82_stats.txt
../epiddrad/clust_83_outfiles/clust_83_stats.txt
[   number variable sum_var   pis sum_pis  \
0       0       80       0  1114       0   
1       1      187     187  1010    1010   
2       2      241     669  1004    3018   
3       3      299    1566   922    5784   
4       4      390    3126  1005    9804   
5       5      474    5496  1081   15209   
6       6      574    8940  1099   21803   
7       7      700   13840  1084   29391   
8       8      727   19656  1137   38487   
9       9      851   27315  1240   49647   
10     10      978   37095  1104   60687   
11     11      964   47699  1038   72105   
12     12     1109   610

In [12]:
# MESSY CODE; INCLUDES NOTES I TOOK WHILE WRITING IT; IGNORE AND DON'T RUN

# The *stats.txt files produced by iPyrad have two blocks of data we want to retrieve: the distribution of SNPs
# (vars and pis) per locus, and the final sample stats summary.
# Start with a list that will contain the first set of data, distribution of SNPs
snp_dist = []

# To just see line numbers within the file:
#with open("epi2brad/clust_84_outfiles/clust_84_stats.txt", "rt") as infile:
#   for line in infile:
#        lines.append(line)
#print(list(enumerate(lines)))

# for now, just leave hard-coded for epifilenames[0]; happens to be clust_threshold=91

#infile=open(ranafilenames[0], 'rt')

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'var  sum_var' in line:
            varline=counter
            print name,varline # not necessary but want to just check
            print line
        elif '## Final' in line:
            endline=counter
            print name,endline
            print line
        else:
            continue
    return varline,endline

#get_line_numbers('../rana2brad/clust_95_outfiles/clust_95_stats.txt')
                
def slice_data(start,end,filename)
    for lines in itertools.islice(filename, start, end-3):
        lines2 = lines.strip().split()
        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        snp_dist.append(lines2)


#print snp_dist
#for name in ranafilenames:
 #   infile=open(name, 'rt')
  #  for lines in itertools.islice(infile, varline, endline-3):
   #     lines2 = lines.strip().split()
    #    lines2.append(name) # eventually change to whatever file it's on
     #   snp_dist.append(lines2)

    #infile.close() # because it's iterated through the whole file already so we need to close and reopen

#snpdist_labels=['number','variable','sum_var','pis', 'sum_pis', 'clust_threshold']
df_snpdist = pd.DataFrame.from_records(snp_dist, columns=snpdist_labels)

print df_snpdist

#df_snpdist.to_csv("./rana2brad_snpdist.csv")


../rana2brad/clust_91_outfiles/clust_91_stats.txt 58
      var  sum_var     pis  sum_pis

../rana2brad/clust_91_outfiles/clust_91_stats.txt 77
## Final Sample stats summary

../rana2brad/clust_90_outfiles/clust_90_stats.txt 149
      var  sum_var     pis  sum_pis

../rana2brad/clust_90_outfiles/clust_90_stats.txt 169
## Final Sample stats summary

../rana2brad/rana2brad_outfiles/rana2brad_stats.txt 241
      var  sum_var     pis  sum_pis

../rana2brad/rana2brad_outfiles/rana2brad_stats.txt 263
## Final Sample stats summary

../rana2brad/clust_86_outfiles/clust_86_stats.txt 335
      var  sum_var     pis  sum_pis

../rana2brad/clust_86_outfiles/clust_86_stats.txt 359
## Final Sample stats summary

../rana2brad/clust_87_outfiles/clust_87_stats.txt 431
      var  sum_var     pis  sum_pis

../rana2brad/clust_87_outfiles/clust_87_stats.txt 452
## Final Sample stats summary

../rana2brad/clust_81_outfiles/clust_81_stats.txt 524
      var  sum_var     pis  sum_pis

../rana2brad/clust_81_outfi

In [111]:
# The SECOND SET of data we're interested in, the summary stats

def get_file_list(directory):
    return glob.glob(directory)

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'state  reads_raw' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#        print line
        else:
            continue
    infile.close()
    return [varline,varline]

def slice_data(start,end,filename):
    sum_stats=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end+12):
        lines2 = lines.strip().split()
        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        sum_stats.append(lines2)
    infile.close()
    return sum_stats
        

def pd_conversion(filename):
    nums = get_line_numbers(filename)
    sum_stats = slice_data(nums[0],nums[1],filename)
    sumstats_labels=['sample', 'state', 'reads_raw', 'reads_passed', 'clust_total', 'clust_hidepth','hetero_est','error_est','reads_consens','loci_assembly','clust_threshold']
    df_sumstats = pd.DataFrame.from_records(sum_stats, columns=sumstats_labels)
    return df_sumstats  

def main():
    directory='../epiddrad/*outfiles/*stats.txt'
    #filename='../rana2brad/clust_95_outfiles/clust_95_stats.txt'
    #directory=filename
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv("./" +filename+ "_sumstats.csv")
    return dfs

print main()

../epiddrad/clust_86_outfiles/clust_86_stats.txt
['Ahah_R0089a', '7', '7869047', '7827474', '1094184', '174267', '0.016359', '0.004850', '144862', '522']
['Ahah_R0089b', '7', '7381407', '7342302', '1120741', '172735', '0.016240', '0.004875', '143577', '519']
['Ahah_R0090', '7', '7112002', '7074480', '980894', '162555', '0.015740', '0.004716', '136395', '539']
['Eant_T6857', '7', '7769532', '7731347', '1044615', '162803', '0.015063', '0.004106', '139000', '17295']
['Eant_T6859a', '7', '7208562', '7173683', '1002139', '155789', '0.014729', '0.004124', '133909', '18178']
['Eant_T6859b', '7', '6156931', '6126704', '806484', '136950', '0.014345', '0.004235', '118510', '17957']
['Ebou_R0153', '7', '6398901', '6370393', '733777', '135476', '0.014729', '0.004412', '116098', '9996']
['Ebou_R0156', '7', '4954025', '4932452', '586887', '116781', '0.013936', '0.004498', '101111', '9688']
['Etri_T6836', '7', '8212760', '8176976', '1064968', '163122', '0.014403', '0.004150', '139517', '16977']
['Etr

In [114]:
# The LAST SET that we're interested in is the LOCUS COVERAGE; this will tell us about missing data and shared loci.

def get_file_list(directory):
    return glob.glob(directory)

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'locus_coverage' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#        print line
        elif '## The distribution' in line:
            endline=counter
            print name,endline
        else:
            continue
    infile.close()
    return [varline,endline]

def slice_data(start,end,filename):
    coverage=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end-3):
        lines2 = lines.strip().split()
        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        coverage.append(lines2)
    infile.close()
    return coverage

def pd_conversion(filename):
    nums = get_line_numbers(filename)
    coverage = slice_data(nums[0],nums[1],filename)
    cov_labels=['number','locus_coverage', 'sum_coverage','clust_threshold']
    df_cov = pd.DataFrame.from_records(coverage, columns=cov_labels)
    return df_cov  

def main():
    directory='../rana2brad/*outfiles/*stats.txt'
    #filename='../rana2brad/clust_95_outfiles/clust_95_stats.txt'
    #directory=filename
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv("./" +filename+ "_sumstats.csv")
    return dfs

print main()

../rana2brad/clust_91_outfiles/clust_91_stats.txt
../rana2brad/clust_89_outfiles/clust_89_stats.txt 53
['1', '0', '0']
['2', '0', '0']
['3', '82472', '82472']
['4', '47612', '130084']
['5', '40668', '170752']
['6', '26442', '197194']
['7', '22948', '220142']
['8', '15615', '235757']
['9', '12171', '247928']
['10', '6342', '254270']
['11', '3898', '258168']
['12', '2222', '260390']
../rana2brad/clust_90_outfiles/clust_90_stats.txt
../rana2brad/clust_89_outfiles/clust_89_stats.txt 53
['1', '0', '0']
['2', '0', '0']
['3', '82268', '82268']
['4', '47754', '130022']
['5', '40740', '170762']
['6', '26436', '197198']
['7', '22942', '220140']
['8', '15546', '235686']
['9', '12148', '247834']
['10', '6336', '254170']
['11', '3878', '258048']
['12', '2238', '260286']
../rana2brad/clust_86_outfiles/clust_86_stats.txt
../rana2brad/clust_89_outfiles/clust_89_stats.txt 53
['1', '0', '0']
['2', '0', '0']
['3', '67784', '67784']
['4', '40103', '107887']
['5', '34179', '142066']
['6', '22632', '164698'

In [109]:
# Now, let's work on merging these dataframes together from each parameter run.
# They're contained within their _outfile folder, so we need to pull them out and merge them somehow
# To make life easier, I just did this in terminal using the following:
# $ mv *outfiles/*.csv .

# Now they're all present within our directory of choice (../rana2brad/*.csv)

df1 = pd.read_csv("../epi2brad/clust_80_stats.txt_sumstats.csv")
df2 = pd.read_csv("../epi2brad/clust_81_stats.txt_sumstats.csv")
df3 = pd.read_csv("../epi2brad/clust_82_stats.txt_sumstats.csv")
df4 = pd.read_csv("../epi2brad/clust_83_stats.txt_sumstats.csv")
df5 = pd.read_csv("../epi2brad/clust_84_stats.txt_sumstats.csv")
df6 = pd.read_csv("../epi2brad/clust_85_stats.txt_sumstats.csv")
df7 = pd.read_csv("../epi2brad/clust_86_stats.txt_sumstats.csv")
df8 = pd.read_csv("../epi2brad/clust_87_stats.txt_sumstats.csv")
df9 = pd.read_csv("../epi2brad/clust_88_stats.txt_sumstats.csv")
df10 = pd.read_csv("../epi2brad/clust_89_stats.txt_sumstats.csv")
df11 = pd.read_csv("../epi2brad/clust_90_stats.txt_sumstats.csv")
df12 = pd.read_csv("../epi2brad/clust_91_stats.txt_sumstats.csv")
df13 = pd.read_csv("../epi2brad/clust_92_stats.txt_sumstats.csv")
df14 = pd.read_csv("../epi2brad/clust_93_stats.txt_sumstats.csv")
df15 = pd.read_csv("../epi2brad/clust_94_stats.txt_sumstats.csv")
df16 = pd.read_csv("../epi2brad/clust_95_stats.txt_sumstats.csv")

sumstats_concat = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16])
sumstats_concat.to_csv("epi2brad_sumstats.csv")